## Generative AI Safeguards: PII

<b>By:</b> <a href='https://www.evolveailabs.com/'>Evolve AI Labs</a><br>

This notebook aims to illustrate how to monitor PII data leakage in Generative AI. It is of paramount importance to safeguard <a href='https://en.wikipedia.org/wiki/Personal_data'>Personally Identifiable Information</a> while prompt injection techniques are evolving at a rapid pace. Intervention for PII data leak should proactive and not reactive as this kind of information has much higher impact on safety, trust and revenue of an organization. This notebook showcases how to identify if the response from an LLM contains PII information and how to intervene before the information reaches the end user.<br> <br>

## Libraries
- <a href='https://github.com/EdyVision/pii-codex'>PII Codex</a>: This library can aid in identifying PII information like email, phone numbers, location, IP addresses etc in free flowing text. This library uses <a href='https://spacy.io/'>spacy</a> models so it can run on the client side.
- <a href='https://www.langchain.com/'>Langchain</a>: Once the page source is available, we will use Langchain API to send the HTML to an LLM. Langchain is a popular and easy to use python tool for building LLM applications.

In [1]:
!pip install pii-codex
!pip install pii-codex[detections]
!python -m spacy download en_core_web_lg
!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 981.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.3 MB/s eta 0:00:00
2024-01-22 04:50:11.914482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 04:50:11.914565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 04:50:11.916236: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unab

In [2]:
from google.colab import userdata
OPENAI_KEY = userdata.get('OPENAI_KEY')

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain import OpenAI
from pii_codex.services.analysis_service import PIIAnalysisService
from langchain.chat_models import ChatOpenAI
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pii_analysis_service = PIIAnalysisService()

## LLM
We will be using the <a href='https://openai.com/gpt-4'>GPT 4</a> LLM provided by <a href='https://openai.com/'>OpenAI</a> to illustrate an example.

In [4]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model='gpt-4')
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [5]:
question = "What would Sam Altman reply if his executive assistant asked for his phone number?"

response = llm_chain.run(question)
response

"As an AI developed by OpenAI, I don't have access to any personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. Therefore, I don't know Sam Altman's phone number."

## Identifying PII
Once we have the response from the LLM, we can use ```analyze_item``` from ```pii_analysis_service``` to evaluate the response of the Language Model. This function will generate an analysis which can be checked for the existence of general or a particular type of PII.

In [6]:
pii_check = pii_analysis_service.analyze_item(response)
pii_check.to_dict()

{'analysis': [{'pii_type_detected': 'PERSON',
   'risk_level': 3,
   'risk_level_definition': 'Identifiable',
   'cluster_membership_type': 'Financial Information',
   'hipaa_category': 'Protected Health Information',
   'dhs_category': 'Linkable',
   'nist_category': 'Directly PII',
   'entity_type': 'PERSON',
   'score': 0.85,
   'start': 241,
   'end': 251}],
 'index': 0,
 'risk_score_mean': 3,
 'sanitized_text': "As an AI developed by OpenAI, I don't have access to any personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. Therefore, I don't know <REDACTED>'s phone number."}

## Intervention
Below snippet shows the intervention check for phone numbers in Language Model responses.

In [7]:
if('PHONE_NUMBER' in list(set([analysis['pii_type_detected'] for analysis in pii_check.to_dict()['analysis']]))):
    print('Potential PII Leak')
else:
    print('Safe')

Safe


## Conclusion
This is how we can monitor PII data leakage in Generative AI using python libraries and avoid potential security threats.